In [ ]:
##### BIGRAM LANGUAGE MODEL FOR DIACRITICS RESTORATION #####

In [1]:
### PREPROCESSING PRO DICTIONARY ###

In [2]:
#otevřít textový soubor - mode "r" i.e. read, uložit do proměnné data veškerý text
with open("corpus_diacritics.txt", "r") as file:
    data_split = file.read().split() #split by měl rozdělit na slova

In [3]:
#rozdělená slova hodit do lower case
data_words_lowered = list(map(lambda x: x.lower(), data_split))

In [4]:
#zbavit se interpunkcí ve strings
import re
data_no_interpunction = list(map(lambda x: re.sub(r'[^\w\s]', '', x) , data_words_lowered))

In [5]:
del data_no_interpunction[109272:] #odstanění anglické části na konci

In [6]:
del data_no_interpunction[0:127] #odstanění anglické části na začátku

In [7]:
data_no_interpunction = sorted(data_no_interpunction)
del data_no_interpunction[0:579]

In [8]:
data_ready = data_no_interpunction

In [9]:
### DICTIONARY KNOWN WORDS ###

In [10]:
from collections import OrderedDict
data_list_dia = list(OrderedDict.fromkeys(data_ready))

In [11]:
def odstranDiakritiku(lst):
    upravenySeznam = []
    seznamPismenek = \
        [["ě", "e"], \
        ["š", "s"], \
        ["č", "c"], \
        ["ř", "r"], \
        ["ž", "z"], \
        ["ý", "y"], \
        ["á", "a"], \
        ["é", "e"], \
        ["ň", "n"], \
        ["ó", "o"], \
        ["ů", "u"], \
        ["ú", "u"], \
        ["ď", "d"], \
        ["ť", "t"], \
        ["í", "i"]]
    for slovo in lst:
        for k in range(len(seznamPismenek)):
            slovo = slovo.replace(seznamPismenek[k][0],seznamPismenek[k][1])
        upravenySeznam.append(slovo)
    return upravenySeznam

In [12]:
#odstraneni diakritiky
lst = data_list_dia
data_list_no_dia = odstranDiakritiku(lst)

In [13]:
from collections import Counter
data_dict_count_dia = Counter(data_ready)

data_list_count_dia =[]
for key in data_dict_count_dia.keys():
    data_list_count_dia.append(data_dict_count_dia[key])

In [14]:
import pandas as pd
df_data = pd.DataFrame([data_list_no_dia, data_list_dia, data_list_count_dia])#, index=["no_dia", "dia", "count_dia"])

df_data_transposed = df_data.transpose()
df_data_transposed = df_data_transposed.rename(columns={0: "no_dia", 1: "dia", 2: "count_dia"})

df = df_data_transposed.astype({"no_dia": object, "dia": object, "count_dia": int})

In [15]:
dictionary_words = df[df['count_dia'] == df.groupby('no_dia')['count_dia'].transform('max')].drop("count_dia",axis=1).set_index("no_dia").transpose().to_dict(orient='records')[0]

/opt/pyenv/versions/3.7.8/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [16]:
### JAZYKOVY MODEL ###

In [17]:
with open("corpus_diacritics.txt", "r") as file:
    content = file.read().split()
#content = [" {}".format(elem) for elem in content]

In [18]:
#rozdělená slova hodit do lower case
data_words_lowered = list(map(lambda x: x.lower(), content))

In [19]:
#zbavit se interpunkcí ve strings
import re
data_no_interpunction = list(map(lambda x: re.sub(r'[^\w\s]', '', x) , data_words_lowered))

In [20]:
del data_no_interpunction[109272:] #odstanění anglické části na konci

In [21]:
del data_no_interpunction[0:127] #odstanění anglické části na začátku

In [22]:
# funkce na vytahovani z elemtu v listu jeden string mimo list
def extractChars(lst):
    return [list(el) for el in lst]
                  
lst = data_no_interpunction
data_format = extractChars(lst)

In [23]:
# nastaveni padding na mezeru misto <s> </s>
from functools import partial
from itertools import chain

from nltk.util import everygrams, pad_sequence
from nltk.lm.preprocessing import pad_both_ends
pad_both_ends = partial(
    pad_sequence,
    pad_left=True,
    left_pad_symbol=" ",
    pad_right=True,
    right_pad_symbol=" ",)

In [24]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, data_format)

In [25]:
from nltk.lm import MLE
lm = MLE(2)

In [26]:
lm.fit(train, vocab)

In [51]:
### INPUT PROCESSING PRO HLEDANI V DICT A OUTPUT GENEROVANI ###

In [27]:
#zpravování inputu do správného codingu

In [28]:
import codecs
fileObj = codecs.open("development_set.txt", "r", "utf-8" )
input_data = fileObj.read() # Returns a Unicode string from the UTF-8 bytes in the file
import ftfy
input_data = ftfy.fix_text(input_data).split()

In [29]:
#rozdělená slova hodit do lower case
input_data_words_lowered = list(map(lambda x: x.lower(), input_data))

In [30]:
#zbavit se interpunkcí ve strings
import re
input_data_no_interpunction = list(map(lambda x: re.sub(r'[^\w\s]', '', x) , input_data_words_lowered))

In [31]:
lst = input_data_no_interpunction
input_data_no_dia = odstranDiakritiku(lst)

In [32]:
### HLAVNI CAST - OUTPUT GENEROVANI ###

In [33]:
def pridejDiakritiku(lst):
    upravenySeznam = []
    seznamPismenek = \
        [["ě", "e"], \
        ["š", "s"], \
        ["č", "c"], \
        ["ř", "r"], \
        ["ž", "z"], \
        ["ý", "y"], \
        ["á", "a"], \
        ["é", "e"], \
        ["ň", "n"], \
        ["ó", "o"], \
        ["ů", "u"], \
        ["ú", "u"], \
        ["ď", "d"], \
        ["ť", "t"], \
        ["í", "i"]]
    for slovo in lst:
        for k in range(len(seznamPismenek)):
            slovo = slovo.replace(seznamPismenek[k][1],seznamPismenek[k][0])
        upravenySeznam.append(slovo)
    return upravenySeznam[0]

In [34]:
seznamPismenek = \
                [["ě", "e"], \
                ["š", "s"], \
                ["č", "c"], \
                ["ř", "r"], \
                ["ž", "z"], \
                ["ý", "y"], \
                ["á", "a"], \
                ["é", "e"], \
                ["ň", "n"], \
                ["ó", "o"], \
                ["ů", "u"], \
                ["ú", "u"], \
                ["ď", "d"], \
                ["ť", "t"], \
                ["í", "i"]]

In [35]:
stringPismenka = "aeiyoutdszcnr"

In [38]:
from nltk.util import bigrams
output = []
rekonstrukce_slova = []
#pro vsechna slova v inputu

for slovo in input_data_no_dia:
    #chci vedet jestli to slovo uz znam a mam v dictu
    if slovo in dictionary_words:
        #pokud ano tak najdu ekvivalent v dictu a jeho value davam rovnou do outputu
        output.append(dictionary_words[slovo])
    else:
        #zpracovavam input slova abych mohl hledat bigrams
        rekonstrukce_slova = []
        lst = slovo
        data_format = extractChars(lst)        
        bigram_format = list(bigrams(pad_both_ends(data_format, n=2)))
        
        # rozkladam na jednotlive bigramy
        for bigram_tupple in bigram_format:

            #vytvarim chars aktualniho bigramu abych mohl hledat v jazykovem modelu
            bigram_letter_first = bigram_tupple[0]
            bigram_letter_second = bigram_tupple[1]
            #osetreni toho, ze jsem musel konvertovat na obyceny string
            if bigram_letter_first == " ":
                bigram_letter_first = bigram_letter_first
            else:
                bigram_letter_first = bigram_letter_first[0]
            
            if bigram_letter_second == " ":
                break #pokud je druhy letter mezera, tak jsem na konci slova
            else:
                bigram_letter_second = bigram_letter_second[0]
            #restoration of diacritics chci delat jenom pro bigramy kde druhe pismeno je v seznamu pismen co maji verzi s diakritikou
            if bigram_letter_second in stringPismenka:
                #kdyz ma pismeno ze seznamu, tak chci vedet jestli takovy bigram existuje v seznamu bigramu jazykoveho modelu
                if lm.score(bigram_letter_second, [bigram_letter_first]) > 0.0:
                    #kdyz takovy bigram znam, tak musim vytvorit komplementarni verzi bigramu s diakritikou
                    bigram_complement_letter_second = pridejDiakritiku(bigram_letter_second)
                    #potrebuju zjistit jestli takovy komplementarni bigram existuje v seznamu bigramu meho jazykoveho modelu
                    if lm.score(bigram_complement_letter_second, [bigram_letter_first]) > 0.0:
                        #kdyz existuje, pak existuji obe a ja si chci vybrat ten s vetsim score tj. pravdepodobnosti
                        if lm.score(bigram_letter_second, [bigram_letter_first])>lm.score(bigram_complement_letter_second, [bigram_letter_first]):
                            #pridavam druhe pismeno z bigram bez dia do rekonstrukted
                            rekonstrukce_slova.append(bigram_letter_second[0])
                        else:
                            #pridavam druhe pismeno z bigram s dia do rekonstrukted

                            rekonstrukce_slova.append(bigram_complement_letter_second[0])
                    else:
                        #nemam komplementarni bigram v seznamu, takze vybiram bez diakritiky do rekonstrukted
                        rekonstrukce_slova.append(bigram_letter_second[0])
                    
                else:
                    #nemame tento bigram v seznamu jazykoveho modelu
                    rekonstrukce_slova.append(bigram_letter_second[0])
                    #pridavam druhe pismeno do rekonstrukted
            else:
                #nema bigram pismeno ze seznamu, druhe pismeno davam do rekonstrukted
                rekonstrukce_slova.append(bigram_letter_second[0])
                
        output.append("".join(rekonstrukce_slova))

In [ ]:
print(output)

In [ ]:
### EVALUACE ###

In [ ]:
#musim vzit vsechno z outputu a jedno podruhym srovnat s inputem ktery ma diakritiku -> pokud se bude rovnat, tak dat na merak +1, pak udelat procento z uspechu a celku

In [40]:
len([i for i, j in zip(output, input_data_no_interpunction) if i == j])/len(input_data_no_interpunction)*100

69.28477402369461